## 14p Final Project (Experiment): Joshua Zempoalteca & Yara Currier-Herzallah

In [1]:
import numpy as np
from numpy import random 
import pandas as pd
import simpleaudio as sa

In [12]:
subjid = 2 # unique value set by experimenter
baseFreq = 1760 # set by experimenter in Hz

randomNumSeed = subjid
startingFreqDiff = 0.1 #default from prompt
numFixedTrials = 5 #default from prompt
numReversals = 4 #default from prompt
maxTrials = 50 #default from prompt
duration = 0.5 
volume = 0.1
silent_interval = 0.25 # silent interval between the two notes
sr = 44100
filename =  'subj'+str(subjid) + 'freq'+str(baseFreq) + '.csv'
completeTrials = 0
reversals = 0

In [13]:
def make_tone(f,duration,sr = 44100,ramp = 200):
    time_vec = np.linspace(0, duration, int(duration*sr)) # The number of samples is the length of time X sampling rate.
    tone = np.sin(f * time_vec  * 2 * np.pi)
    risingramp = np.linspace(0,1,ramp)
    descendingramp = np.linspace(1,0,ramp)
    tone[0:ramp] = tone[0:ramp]*risingramp
    tone[-ramp:] = tone[-ramp:]*descendingramp
    return tone
def play_sound(tone,volume = 0.05,sr = 44100):
    tone  = tone*32768 / np.max(np.abs(tone)) # scale to the range of the sound card.  
    tone = volume*tone # FOR SAFETY.  PLEASE LIMIT THE MAXIMIM VOLUME! 
    tone  = tone.astype(np.int16) # convert to 16 bit integers. 
    play_obj = sa.play_buffer(tone , 1, 2, sr) # i created an object here. 
    play_obj.wait_done() # tells python to wait for the sound to finish before going any further.  
def randomize_trials(ctrials,ncond):
    trials = np.zeros(ctrials*ncond)
    for j in range(ncond):
        trials[j*ctrials:(j+1)*ctrials] = j*np.ones(ctrials) # an array with ntrials ones
    shuffle = random.permutation(ctrials*ncond) # get a random order of trials
    trials = trials[shuffle] # permute trial order 
    return trials

In [14]:
rng = random.default_rng(seed = subjid)

In [15]:
df = startingFreqDiff
A_note = make_tone(baseFreq, duration)
fB = baseFreq + (baseFreq * df)
B_note = make_tone(fB,duration)
silence = np.zeros(int(silent_interval*sr))

trial_order = randomize_trials(maxTrials, 2)

In [16]:
correct_response = np.zeros(maxTrials, dtype = int) #empty array to hold the trial labels
trial_response = np.array(np.zeros(maxTrials),dtype = int) #empty array to hold the responses of trype string
A_notes = np.zeros(maxTrials)
freq_diffs = np.zeros(maxTrials, dtype = float)

In [17]:
for trial in range(numFixedTrials): 
    if trial_order[completeTrials] == 0: 
        stimulus = np.concatenate((A_note,silence,B_note))
        play_sound(stimulus,volume=volume)
        correct_response[trial] = 2
    else:
        stimulus = np.concatenate((B_note,silence,A_note))
        play_sound(stimulus,volume=volume)
        correct_response[trial] = 1
    response_check = False 
    while response_check == False:
        response = input('Is the first or 2nd note higher') 
        if (response =='1') | (response == '2'): 
            response_check = True 
            trial_response[trial] = int(response)
    A_notes[completeTrials] = baseFreq
    freq_diffs[completeTrials] = df
    completeTrials += 1
while completeTrials < maxTrials: 
    if trial_order[completeTrials] == 0: 
        stimulus = np.concatenate((A_note,silence,B_note))
        play_sound(stimulus,volume=volume)
        correct_response[completeTrials] = 2
    else:
        stimulus = np.concatenate((B_note,silence,A_note))
        play_sound(stimulus,volume=volume)
        correct_response[completeTrials] = 1
    response_check = False  
    while response_check == False:
        response = input('Is the first or 2nd note higher')
        if (response =='1') | (response == '2'): 
            response_check = True
    trial_response[completeTrials] = int(response) 
    A_notes[completeTrials] = baseFreq
    freq_diffs[completeTrials] = df
    if (trial_response[completeTrials] == correct_response[completeTrials]) & (trial_response[completeTrials - 1] == correct_response[completeTrials - 1]):
        if freq_diffs[completeTrials] == freq_diffs[completeTrials - 1]:
            df = df/(np.sqrt(2))
    elif trial_response[completeTrials] != correct_response[completeTrials]: 
        df = df*(np.sqrt(2))
        reversals +=1
    if reversals == numReversals + 1: 
        break
    fB = A_note + (A_note + df)          
    completeTrials += 1

print(str(baseFreq)+' Hz task complete.')


1760 Hz task complete.


In [18]:
data = pd.DataFrame(columns = ['Frequency of A note', 'Condition','Response', 'Frequency Difference']) #create a data frame
data['Frequency of A note'] = A_notes
data['Frequency Difference'] = freq_diffs
data['Condition'] = correct_response #save trial order
data['Response'] = trial_response  #save response 
data.to_csv(filename, index= False)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Function to compute the threshold value
def compute_threshold(reversals):
    return np.mean(reversals[1:5])

# Load the data for all participants
participants_data = [
    {'participant_id': 1, 'data': {'trial_number': [1, 2, 3, 4, 5], 'frequency_difference': [0.5, 0.7, 0.6, 0.4, 0.8]}},
    {'participant_id': 2, 'data': {'trial_number': [1, 2, 3, 4, 5], 'frequency_difference': [0.3, 0.2, 0.4, 0.5, 0.6]}},
    {'participant_id': 3, 'data': {'trial_number': [1, 2, 3, 4, 5], 'frequency_difference': [0.9, 0.8, 0.7, 0.6, 0.5]}},
]

# Plotting the frequency difference for each participant
for participant_data in participants_data:
    trial_number = participant_data['data']['trial_number']
    frequency_difference = participant_data['data']['frequency_difference']
    training_trials = [True, False, False, True, False]  # Example training trials, replace with your data
    
    plt.plot(trial_number, frequency_difference, 'bo', label='All Trials')
    plt.plot(np.array(trial_number)[training_trials], np.array(frequency_difference)[training_trials], 'ro', label='Training Trials')
    
    # Add a horizontal line indicating the threshold
    threshold = compute_threshold(frequency_difference)
    plt.axhline(y=threshold, color='g', linestyle='--', label='Threshold')
    
    plt.xlabel('Trial Number')
    plt.ylabel('Frequency Difference')
    plt.title('Participant {}'.format(participant_data['participant_id']))
    plt.legend()
    plt.grid(True)
    plt.show()

# Compute the threshold value for each frequency and create a bar graph
frequencies = ['Freq1', 'Freq2', 'Freq3']  # Replace with your frequencies
thresholds = []

for participant_data in participants_data:
    frequency_difference = participant_data['data']['frequency_difference']
    threshold = compute_threshold(frequency_difference)
    thresholds.append(threshold)

x = np.arange(len(frequencies))
plt.bar(x, thresholds)
plt.xlabel('Frequency')
plt.ylabel('Threshold')
plt.title('Threshold for each Frequency')
plt.xticks(x, frequencies)
plt.show()
